In [1]:
import zipfile
with zipfile.ZipFile("dataset/runs_data.zip","r") as zip_ref:
    zip_ref.extractall("dataset")

In [2]:
import pandas as pd
df = pd.read_csv("dataset/runs_data.csv")
print(df.shape)

(1162752, 6)


In [3]:
df.head()

,Unnamed: 0,time,measured_position,demand_current,operation_mode,run_no
0,0,0.000,-0.0003,-1.848,1,0
1,1,0.006,0.0018,-1.876,1,0
2,2,0.012,-0.0003,-1.836,1,0
3,3,0.018,0.0003,-1.852,1,0
4,4,0.024,-0.0009,-1.841,1,0


# Part 1

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
mode_description = pd.read_excel('Operation_Mode_Description.xlsx')
mode_description

,Operation Mode,Description of Mode
0,1,Normal operation
1,2,Slow opening of the grippers
2,3,The motor has too high velocity
3,4,The motor applies too much force when placing ...
4,5,The component was rotated by 90 degrees before...


In [6]:
df_eda = df.merge(mode_description, left_on= 'operation_mode', right_on = 'Operation Mode').rename(columns = {'Unnamed: 0':'obs_num'})\
        [['run_no','obs_num', 'time', 'measured_position', 'demand_current','Description of Mode','operation_mode']]
df_eda.head()

,run_no,obs_num,time,measured_position,demand_current,Description of Mode,operation_mode
0,0,0,0.000,-0.0003,-1.848,Normal operation,1
1,0,1,0.006,0.0018,-1.876,Normal operation,1
2,0,2,0.012,-0.0003,-1.836,Normal operation,1
3,0,3,0.018,0.0003,-1.852,Normal operation,1
4,0,4,0.024,-0.0009,-1.841,Normal operation,1


In [7]:
df_eda.groupby('run_no')['operation_mode'].first().astype("category").to_frame()['operation_mode'].value_counts()

4    572
2    504
1    502
5    351
3    342
Name: operation_mode, dtype: int64

In [8]:
# df_eda.to_excel('df.xlsx')

## Exploratory data analysis
The test experiment gathered timeseries data of displacement, velocity, and force signal reported by the linear motor during the assembly step. However, here you only get the data for the current demanded by the linear motor controller and the measured position of the component.         

The dataset consists of 2271 different runs. 

![All](visual1.png)

Each run has 512 observations with each observation having a time value and the values for the current and position at that time. The point in time where the motor is triggered could vary from one run to another.              
![All](one.png)      

## Create candidate variables for each run   
1. displacement of component:      
    - Movement is linear, one dimensional and follows a pattern.           
    - Time interval for recording is constant.  
    
          
2. velocity for current          
    - Current demanded varies as component position changes. 
    - Position changes have two critical phases. Current at each phase has different characteristics.
                      
                      
3. force signal, which is not given in the data -- Unavailable  
    - Forces signal may be a significant factor for classify operation mode 4: applies too much force. Thus, may hinder model performance.

In [9]:
def create_candidate_variables(inputFile, run_no = 'run_no'):
    '''create variables for each run'''
    import pandas as pd
    
    # a vacant dataset to store features
    data = pd.DataFrame(index = df[run_no].unique())
    
    def get_summary( column = 'displacement',method = ['max','min','std'],df= df):
        '''group by run no'''
        return df.groupby(run_no).agg({column: method})

    #1 displacement
    df['lag_position'] = df.groupby('run_no')['measured_position'].shift(1)
    df['displacement'] = df['measured_position'] - df['lag_position']
    data = data.join(get_summary(column = 'displacement', method = ['max']))

    #2 velocity
    data = data.join(get_summary(column = 'demand_current', method = ['std']))

    #3 position
    data = data.join(get_summary(column = 'measured_position', method = ['last']))
    
    #4 position - current 
    phase1_lower = 90
    phase1_upper = 120
    p1 = df[df['measured_position'].between(phase1_lower, phase1_upper)].rename(columns = {'demand_current':'demand_current_p1'})
    data = data.join(get_summary(column = 'demand_current_p1', df =p1))
    
    phase2_lower = 134
    phase2_upper = 160
    p2 = df[df['measured_position'].between(phase2_lower, phase2_upper)].rename(columns = {'demand_current':'demand_current_p2'})
    data = data.join(get_summary(column = 'demand_current_p2', df =p2))

    
    return data

In [10]:
data= create_candidate_variables(df, run_no = 'run_no')
data.join(df.groupby('run_no')['operation_mode'].first()).sample(5)

,"(displacement, max)","(demand_current, std)","(measured_position, last)","(demand_current_p1, max)","(demand_current_p1, min)","(demand_current_p1, std)","(demand_current_p2, max)","(demand_current_p2, min)","(demand_current_p2, std)",operation_mode
1616,3.2349,2.370276,0.0800,6.316,-4.500,1.680585,4.5,-3.591,2.090633,4
1728,3.2642,2.210400,0.0766,10.000,-7.054,2.378287,4.5,-3.528,1.971189,4
1122,3.2632,2.709882,0.0491,4.291,-4.168,1.535719,10.0,-3.881,3.271743,3
1346,3.9929,2.500654,0.0845,5.228,-4.024,1.303428,10.0,-4.169,3.307137,3
925,6.0663,2.222245,0.1068,5.000,-2.912,1.581041,5.0,-5.000,2.197867,2


#### 9 variables have been created

## Try various ML methods   

Considersions for selecting ML methods:
- Use KNN as baseline model      
- Multiclass classfication             
- From EDA, the classes are well separated

In [11]:
from sklearn.model_selection import train_test_split
X = data
y = df.groupby('run_no')['operation_mode'].first()

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.3, random_state=55)

In [12]:
######## choose classfication model
# Baseline model
from sklearn.neighbors import KNeighborsClassifier
# The classes are well-separated + Multiclass classification 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

names = ["Nearest Neighbors", "LDA", "QDA", 
         "Decision Tree", "Random Forest" ]

classifiers = [
    KNeighborsClassifier(3),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    DecisionTreeClassifier(random_state=0),
    RandomForestClassifier(random_state=0)
    ]

# iterate over classifiers
for name, clf in zip(names, classifiers):
    pred = clf.fit(X_train, y_train).predict(X_test)
    print(name )
    print( confusion_matrix(y_test, pred).T)
    print( classification_report(y_test, pred, digits=3))

Nearest Neighbors
[[145   4   0   2   0]
 [  2 151   0   0   0]
 [  0   0 102   0   0]
 [  0   1   0 168   0]
 [  0   0   0   0 107]]
              precision    recall  f1-score   support

           1      0.960     0.986     0.973       147
           2      0.987     0.968     0.977       156
           3      1.000     1.000     1.000       102
           4      0.994     0.988     0.991       170
           5      1.000     1.000     1.000       107

   micro avg      0.987     0.987     0.987       682
   macro avg      0.988     0.989     0.988       682
weighted avg      0.987     0.987     0.987       682

LDA
[[147   6   0  22   0]
 [  0 150   0   0   0]
 [  0   0 102   0   0]
 [  0   0   0 148   0]
 [  0   0   0   0 107]]
              precision    recall  f1-score   support

           1      0.840     1.000     0.913       147
           2      1.000     0.962     0.980       156
           3      1.000     1.000     1.000       102
           4      1.000     0.871     0.


For this case, our objective is to classify different modes of operation and failure types. Hence weighted avg F-1 score can be used to evaluate overall multiclass classification perfomance (Accuracy score can also be a good and simple choice. ) In addition, it is important to not misclassify failed run as normal. So I also want to choose a model with highest precison rate of mode 1.            

         
#### I will choose Random Forest model as it has the highest weightes avg F1 score (99.3%) and highest precision for operation mode1 (99.3%).

In [13]:
# save my classfier model for later implement
my_clf =  RandomForestClassifier(random_state=0).fit(X, y)
#!pip install joblib
from joblib import dump, load
dump(my_clf, 'my_classifer.pkl') 

['my_classifer.pkl']

#### Next steps if have more time:
- Acqurie data on force signal
- Build more expert variables by talking with domain experts. The goal is to minimize the work of my algorithm.
- Minimize dimensionality
- Fine ture parameters used in my decision tree algorithm
- Clafity on user requirements, how the algorithm will be used

## Implement

In [14]:

def create_variables(inputFile, run_no = 'run_no'):
    '''create variables for each run'''
    import pandas as pd
    df = pd.read_csv(inputFile)

    # a vacant dataset to store features
    data = pd.DataFrame(index = df[run_no].unique())

    def get_summary( column = 'displacement',method = ['max','min','std'],df= df):
        '''group by run no'''
        return df.groupby(run_no).agg({column: method})

    #1 displacement
    df['lag_position'] = df.groupby(run_no)['measured_position'].shift(1)
    df['displacement'] = df['measured_position'] - df['lag_position']
    data = data.join(get_summary(column = 'displacement', method = ['max']))

    #2 velocity
    data = data.join(get_summary(column = 'demand_current', method = ['std']))

    #3 position
    data = data.join(get_summary(column = 'measured_position', method = ['last']))

    #4 position - current 
    phase1_lower = 90
    phase1_upper = 120
    p1 = df[df['measured_position'].between(phase1_lower, phase1_upper)].rename(columns = {'demand_current':'demand_current_p1'})
    data = data.join(get_summary(column = 'demand_current_p1', df =p1))

    phase2_lower = 134
    phase2_upper = 160
    p2 = df[df['measured_position'].between(phase2_lower, phase2_upper)].rename(columns = {'demand_current':'demand_current_p2'})
    data = data.join(get_summary(column = 'demand_current_p2', df =p2))

    return data

def classify(data):
    X =data
    import joblib
    model = joblib.load('my_classifer.pkl')
    y = model.predict(X)
    return y

    
def classify_operation_mode(inputFile):
    '''standalone module '''
    data = create_variables(inputFile)
    result = classify(data)
    
    print('The run has been classified as', result)

In [18]:
# test run has same format as df. It has two run_no: 300 and 1300
# 300 belongs to mode1, 1300 belongs to mode 3
pd.read_csv('test_input.csv').sample(2)

,Unnamed: 0,time,measured_position,demand_current,operation_mode,run_no
852,340,2.040,0.2899,-2.951,3,1300
152,152,0.912,110.7158,1.596,1,300


In [16]:
test = df[df['run_no']==10]
classify_operation_mode('test_input.csv')

The run has been classified as [1 3]


### Part 2

As mentioned, the data provided here was gathered through a test experiment by intentionally inducing some failure modes. The number of runs for different modes of operation are more or less similar. In a real assembly line, the data mainly comes from the normal modes of operation as failure modes are rare. Moreover, we do not exactly know what kind of failures occur and with what characteristics they occur.              

        

To summarize above statement, the challenges in gathering data from a real assembly line are that:
1. Data is labled but imbalanced between normal mode and failure mode(no matter which type). 
2. Data is unlabled within failure modes and getting data labled is expensive.

If our objective is to classfiy normal operation mode and failure operation mode. Since that data is labeld we can still build a supervised ML. Just we have to deal with imbalanced dataset. The major technique to deal with imbalanced dataset is resampling method. A flow is summarised as below.       

![All](imb.png)
             
If our objective is to classify among different failure modes. Then it's more challenging.           
Firstly, run a unsupervised method(clustering) to perform data exploration/find strcture in data, in order to learn what kind of failrues occur. We will need to have a domain expert study the clustering result to decide if the structure is useful. 


![All](cluster.png)

Then, semi-supervised learning is a good choice as less costly to full labeled supervised learning.     
- Semi-supervised learning algorithms use both labeled and unlabeled instances in the training process, they can produce classifiers that achieve better performance than completely supervised learning algorithms that have only a small amount of labeled data available for training.  


As for imbalance, dealing with class imbalance problem in semi-supervised machine learning has been a big topic. Numerous research has been going on in this domain. For example,    

- Stanescu, Ana, Caragea, Doina, Stanescu, Ana. An empirical study of ensemble-based semi-supervised learning approaches for imbalanced splice site datasets. BMC systems biology. 2015;9 suppl 5(Suppl 5):S1-S1. doi:10.1186/1752-0509-9-S5-S1

- Li, Fengqi, Yu, Chuang, Yang, Nanhai, Xia, Feng, Li, Guangming, Kaveh-Yazdy, Fatemeh. Iterative Nearest Neighborhood Oversampling in Semisupervised Learning from Imbalanced Data. December 2013.   

In addition, this answer helps me understand more about this problem.
https://datascience.stackexchange.com/questions/38796/unbalanced-training-data-for-different-classes         
" The reason we want to have a balanced datset is that when the data is strongly imbalanced towards one class, the model may resort to predicting the majority class whenever there's a doubt. This will become a big issue if detection of the minority class is particularly important. Otherwise, resampling datasets in order to reach balanced distributions is a common practice that sometimes improves classification performance. In short, it depends on what is the objective of the classification."                     

